### Import des modules 

In [48]:
# Selection
from sklearn.model_selection import (
    GridSearchCV, 
)
from sklearn.pipeline import Pipeline


# Preprocess
from sklearn.preprocessing import StandardScaler

# Modèles
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import sys
import importlib
import numpy as np

sys.path.append("..")
import config

importlib.reload(config)  # Ensure we get the latest TARGET value
from config import TARGET

### Optimisation et interprétation du modèle

A réaliser :
* Reprennez le meilleur algorithme que vous avez sécurisé via l'étape précédente, et réalisez une GridSearch de petite taille sur au moins 3 hyperparamètres.
* Si le meilleur modèle fait partie de la famille des modèles à arbres (RandomForest, GradientBoosting) alors utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle. Sinon, utilisez la méthode Permutation Importance de sklearn.

In [49]:
import pandas as pd

building_consumption = pd.read_csv("../assets/building_consumption_cleaned.csv")
building_consumption.head()
building_consumption.columns

Index(['NumberofBuildings', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ENERGYSTARScore', 'TotalGHGEmissions', 'NumberOfUseTypes', 'SteamUse',
       'NaturalGas', 'Electricity', 'PrimaryPropertyType_0',
       'PrimaryPropertyType_1', 'PrimaryPropertyType_2',
       'PrimaryPropertyType_3', 'PrimaryPropertyType_4',
       'BuildingType_Nonresidential COS', 'BuildingType_Nonresidential WA',
       'BuildingType_SPS-District K-12', 'NumberofFloors_quintile_Q2',
       'NumberofFloors_quintile_Q3', 'NumberofFloors_quintile_Q4',
       'NumberofFloors_quintile_Q5', 'EraBuilt_1951-1970',
       'EraBuilt_1971-1990', 'EraBuilt_1991-2010', 'EraBuilt_Post-2010',
       'EraBuilt_≤1930'],
      dtype='object')

In [50]:
# Préprocessor : toutes les features sont numériques et encodées
preprocessor = StandardScaler()

# Pipeline RandomForest
pipeline_rf = Pipeline(
    [("model", RandomForestRegressor(random_state=42))]
)

# Hyperparamètres à tester
param_grid = {
    "model__n_estimators": [100, 300, 500],  # nombre d'arbres
    "model__max_depth": [None, 5, 10, 20],  # profondeur max
    "model__min_samples_split": [2, 5, 10],  # nb min d'échantillons pour splitter
}

grid_search = GridSearchCV(
    pipeline_rf, param_grid, cv=5, scoring="r2", n_jobs=-1, verbose=1
)

# On sépare X et y
X = building_consumption.drop(columns=[TARGET])
y = building_consumption[TARGET]

# Lancement de la recherche
grid_search.fit(X, y)

print("Meilleurs hyperparamètres :", grid_search.best_params_)
print("Meilleur R² :", grid_search.best_score_)

# Feature importance
best_model = grid_search.best_estimator_.named_steps["model"]
importances = best_model.feature_importances_
feature_names = X.columns



feat_imp_df = pd.DataFrame(
    {"feature": feature_names, "importance": importances}
).sort_values(by="importance", ascending=False)

print(feat_imp_df.head(10))  # top 10 features

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Meilleurs hyperparamètres : {'model__max_depth': 20, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Meilleur R² : 0.3416354691825433
                       feature  importance
0            NumberofBuildings    0.412141
2       PropertyGFABuilding(s)    0.279959
3              ENERGYSTARScore    0.077779
19  NumberofFloors_quintile_Q5    0.038474
10       PrimaryPropertyType_2    0.025625
6                   NaturalGas    0.023498
5                     SteamUse    0.018277
11       PrimaryPropertyType_3    0.016532
4             NumberOfUseTypes    0.015667
9        PrimaryPropertyType_1    0.015127
Meilleurs hyperparamètres : {'model__max_depth': 20, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Meilleur R² : 0.3416354691825433
                       feature  importance
0            NumberofBuildings    0.412141
2       PropertyGFABuilding(s)    0.279959
3              ENERGYSTARScore    0.077779
19 

### Prédiction sur de nouveaux bâtiments

Maintenant que nous avons un modèle optimisé, voyons comment l'utiliser pour faire des prédictions sur des bâtiments inconnus.

In [51]:
def predict_building_energy(building_data, model=grid_search.best_estimator_):
    """
    Prédit la consommation énergétique d'un ou plusieurs bâtiments
    
    Args:
        building_data: DataFrame ou dict avec les caractéristiques du/des bâtiment(s)
        model: Modèle entraîné (par défaut le meilleur modèle de GridSearch)
    
    Returns:
        Prédiction(s) de consommation énergétique
    """
    import pandas as pd
    
    # Convertir en DataFrame si c'est un dictionnaire
    if isinstance(building_data, dict):
        building_data = pd.DataFrame([building_data])
    
    # Vérifier que toutes les features nécessaires sont présentes
    required_features = X.columns.tolist()
    missing_features = set(required_features) - set(building_data.columns)
    
    if missing_features:
        raise ValueError(f"Features manquantes: {missing_features}")
    
    # S'assurer que les colonnes sont dans le bon ordre
    building_data = building_data[required_features]
    
    # Faire la prédiction
    prediction = model.predict(building_data)
    
    return prediction

# Afficher les features requises pour référence
print("Features requises pour la prédiction:")
print(X.columns.tolist())
print(f"\nNombre total de features: {len(X.columns)}")

Features requises pour la prédiction:
['NumberofBuildings', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'ENERGYSTARScore', 'NumberOfUseTypes', 'SteamUse', 'NaturalGas', 'Electricity', 'PrimaryPropertyType_0', 'PrimaryPropertyType_1', 'PrimaryPropertyType_2', 'PrimaryPropertyType_3', 'PrimaryPropertyType_4', 'BuildingType_Nonresidential COS', 'BuildingType_Nonresidential WA', 'BuildingType_SPS-District K-12', 'NumberofFloors_quintile_Q2', 'NumberofFloors_quintile_Q3', 'NumberofFloors_quintile_Q4', 'NumberofFloors_quintile_Q5', 'EraBuilt_1951-1970', 'EraBuilt_1971-1990', 'EraBuilt_1991-2010', 'EraBuilt_Post-2010', 'EraBuilt_≤1930']

Nombre total de features: 25


#### Exemple 1: Prédiction pour un bâtiment de bureau typique

In [52]:
# For a single building
new_building = pd.DataFrame(
    [
        {
            "NumberofBuildings": 1,
            "PropertyGFAParking": 0,
            "PropertyGFABuilding(s)": 56228,
            "ENERGYSTARScore": 95,
            "NumberOfUseTypes": 1,
            "SteamUse": 0,
            "NaturalGas": 1,
            "Electricity": 1,
            # Encodage PrimaryPropertyType (placeholders, dépend de ton encoder exact)
            "PrimaryPropertyType_0": 0,
            "PrimaryPropertyType_1": 0,
            "PrimaryPropertyType_2": 0,
            "PrimaryPropertyType_3": 0,
            "PrimaryPropertyType_4": 0,
            # Encodage BuildingType
            "BuildingType_Nonresidential COS": 0,
            "BuildingType_Nonresidential WA": 0,
            "BuildingType_SPS-District K-12": 1,
            # NumberofFloors quintiles
            "NumberofFloors_quintile_Q2": 1,
            "NumberofFloors_quintile_Q3": 0,
            "NumberofFloors_quintile_Q4": 0,
            "NumberofFloors_quintile_Q5": 0,
            # EraBuilt
            "EraBuilt_1951-1970": 1,
            "EraBuilt_1971-1990": 0,
            "EraBuilt_1991-2010": 0,
            "EraBuilt_Post-2010": 0,
            "EraBuilt_≤1930": 0,
        }
    ]
)
prediction = predict_building_energy(new_building)
print(f"Prédiction de consommation énergétique pour le nouveau bâtiment: {prediction[0]:.2f} kBtu")

Prédiction de consommation énergétique pour le nouveau bâtiment: 3133.07 kBtu
